## 28.3.2021 ##
### First draft of abstract ###
In our first meeting with the group, we discussed about the topic and created a first draft of our abstract.
For this, everyone wrote a draft and we combined our results in the end.

## 29.3.2021 ##
### Revision of abstract ###
In order to comply with the structure shown in https://www.emerald.com/insight/content/doi/10.1108/JD-02-2020-0030/full/html, I revised and restructured our abstract and uploaded the new version the next day, after the other group members agreed with the text.

## 3.4.2021 ##
### First draft of Data Management Plan (DMP) ###
In a group meeting, we created a DMP on the platform Argos (https://argos.openaire.eu/) and divided the further work on it (the description of the two datasets) between us.

## 4.4.2021 ##
### Adding small parts to DMP ###
I concentrated on the parts 6 and 7 of the datasets. When we were all done with our part of the work, we created the first version of our DMP: https://doi.org/10.5281/zenodo.4663016

## 6.4.2021 ##
### Revising first version of DMP ###
Reading the first version of our DMP, I noticed some issues that we revised and updated, creating this version: https://doi.org/10.5281/zenodo.4665853

## 10.4.2021 ##
### Literature review ###
Based on a Google Scholar search with the keywords "wrong doi" "invalid doi" and "doi error", I started with the literature review for the project. I continued the search following the cited references that seemed to contain information about error classes occuring in DOI names and (automatic) methods for fixing and cleaning the DOIs. Especially Xu, S., Hao, L., An, X. et al. Types of DOI errors of cited references in Web of Science with a cleaning method. Scientometrics 120, 1427–1437 (2019). https://doi.org/10.1007/s11192-019-03162-4 (which we already found before) seems to be useful.